In [ ]:
import scipy.io.wavfile as wavefile

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
rate, data = wavefile.read(filename='./ChillingMusic.wav')

In [ ]:
data.shape

In [ ]:
data[:10]

In [ ]:
data.shape[0]/rate # which means rate number of values together makes up 1 sec of the audio.

In [ ]:
def next_batch(batch_size, x, y):
    prev = 0
    for i in range(batch_size , x.shape[0], batch_size):
        yield x[prev:i], y[prev:i]
        prev = i

In [ ]:
no_steps = 30
n_features = 2
learning_rate = 0.001
n_epochs = 1500
batch_size = 40
depth = 3
n_neurons = 100
n_outputs = n_features

In [ ]:
x=[]
_y=[]

In [ ]:
startix = 0
endix = no_steps

while endix+1 < 5*rate: #data for 5 secs.
    x.append(data[startix:endix])
    _y.append(data[startix+1:endix+1])
    startix+=1
    endix+=1

In [ ]:
x = np.array(x)
_y = np.array(_y)

In [ ]:
x.shape, _y.shape

In [ ]:
scaling_factor = x.max()

In [ ]:
x, _y = x/scaling_factor, _y/scaling_factor

In [ ]:
tf.reset_default_graph()

In [ ]:
X = tf.placeholder(shape=[None, no_steps, n_features], dtype=tf.float32)
y = tf.placeholder(shape=[None, no_steps, n_features], dtype=tf.float32)

In [ ]:
with tf.variable_scope('RNN', initializer=tf.contrib.layers.variance_scaling_initializer()):
    cells = [tf.contrib.rnn.LSTMCell(dtype=tf.float32,num_units=n_neurons,
                                     activation=tf.nn.relu,use_peepholes=True) for i in range(2)]
    rnn_layers = tf.contrib.rnn.MultiRNNCell(cells=cells,state_is_tuple=True)
    
    rnn_outputs, states = tf.nn.dynamic_rnn(cell=rnn_layers,dtype=tf.float32,inputs=X)
    stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons])
    dense_outputs= tf.layers.dense(inputs=stacked_rnn_outputs,units=n_outputs, activation=None)
    weight = tf.Variable(np.random.random(size=(1,n_outputs)), dtype=tf.float32)
    outputs = tf.reshape(dense_outputs,[-1, no_steps, n_outputs])

In [ ]:
with tf.name_scope('loss'):
    loss = 1/2*tf.sqrt(tf.reduce_mean(tf.square(outputs*weight - y)))

In [ ]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for x_batch, y_batch in next_batch(batch_size,x,_y):
            sess.run(training_op, feed_dict={X:x_batch, y:y_batch})
        mse = loss.eval(feed_dict = {X:x[:100], y:_y[:100]})
        if not epoch %100:
            print('epoch',epoch,'MSE:',mse)
        
    save_path = saver.save(sess,'./music_generator.ckpt')            

In [ ]:
restore = tf.train.Saver()

In [ ]:
rate*3

In [ ]:
cnt = 0
nval = rate*3
music_gen = [[1000,1000] for i in range(no_steps)]

In [ ]:
with tf.Session() as sess:
    init.run()
    restore.restore(sess, save_path)
    while cnt < nval:
        temp = np.array(music_gen)
        music = outputs.eval(feed_dict = {X:temp[cnt:].reshape(-1,no_steps,n_features)})[0][-1]
        music_gen.append([int(m) for m in music])
        cnt+=1
        if not cnt%1000:
            print(cnt, music_gen[-1])

In [ ]:
my_music = np.array(music_gen)

In [ ]:
wavefile.write('./my_music.wav',rate, my_music)